# Libraries

In [1]:
import numpy as np
np.set_printoptions(suppress=True)

import pandas as pd

import scipy
import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')

from scipy import optimize

# Methods

In [2]:
def auc(y, y_hat):
    n1 = np.sum(y == 1)
    n2 = np.sum(y == 0)
    
    R1 = np.sum(stats.rankdata(y_hat.values)[y.values == 1])
    
    U = R1 - (n1*(n1+1))/2
    
    return(U/(n1*n2))

In [3]:
def disparate_impact(y_hat, s):
    priv = np.mean(y_hat.values[s.values == 0])/np.mean(y_hat.values[s.values == 1])
    disc = np.mean(y_hat.values[s.values == 1])/np.mean(y_hat.values[s.values == 0])
    
    return np.min([priv, disc])

In [4]:
def equality_of_opportunity(y_hat, y, s, out = 1):
    eoo = np.mean(y_hat.values[(s.values == 1) & (y.values == out)]) / np.mean(y_hat.values[(s.values == 0) & (y.values == out)])
    
    return(eoo)

In [5]:
def sigmoid(w, X):
    return(1/(1 + np.exp(-np.sum(w*X, axis=1))))

In [6]:
def logistic_loss(w, X, y):
    y_hat = sigmoid(w, X).values
    
    # NUMERICAL UNDERFLOW AND OVERFLOW
    y_hat[y_hat == 0] = 0.00001
    y_hat[y_hat == 1] = 0.99999
    
    return -np.mean(y.values * np.log2(y_hat) + (1 - y.values) * np.log2(1 - y_hat))

In [7]:
def performance_score(w, X, y, s):
    X_d = X.loc[s == 1, :]
    X_p = X.loc[s == 0, :]
    
    y_d = y[s == 1]
    y_p = y[s == 0]
    
    y_hat = sigmoid(w, X)
    y_d_hat = sigmoid(w, X_d)
    y_p_hat = sigmoid(w, X_p)
    
    print(f'AUC: {auc(y, y_hat)}, AUC Disc: {auc(y_d, y_d_hat)}, AUC Priv: {auc(y_p, y_p_hat)}')
    print(f'LL: {logistic_loss(w, X, y)}, LL Disc: {logistic_loss(w, X_d, y_d)}, LL Priv: {logistic_loss(w, X_p, y_p)}')
    print(f'Disparate impact {disparate_impact(y_hat, s)}')

# Kalai Smorodinsky Solution

In [8]:
class KS_fair_solution:
    
    def __init__(self, normalize=True):
        self.normalize = normalize
        
        return
    
    # ----- HELP FUNCTIONS -----
    def _logistic_loss(self, y, y_hat):
        '''
        Calculate Logistic Loss
        
        Parameters:
        y : numpy.array
            Vector of true values of the outcome
        y_hat : numpy.array
            Vector of predicted values of the outcome
            
        Returns:
        score : ndarray
            Logistic Loss
        '''
        
        return -np.mean(y.values * np.log2(y_hat.values) + (1 - y.values) * np.log2(1 - y_hat.values))
    
    def _disparate_impact(self, y_hat, s):
        '''
        Calculate Disparate Impact
        
        Parameters:
        y_hat : numpy.array
            Vector of predicted values of the outcome
            
        Returns:
        score : ndarray
            Disparate Impact
        '''
        
        priv = np.mean(y_hat.values[s.values == 0])/np.mean(y_hat.values[s.values == 1])
        disc = np.mean(y_hat.values[s.values == 1])/np.mean(y_hat.values[s.values == 0])

        return np.min([priv, disc])
    
    def _sigmoid(self, X):
        return(1/(1 + np.exp(-np.sum(self.w * X, axis=1))))
    
    # ----- OPERATIONS -----
    def _prepare_ks(self):
        '''
        Calculate DataFrame where each attribute is represented as a row, 
        with columns representing Disparate Impact and Logistic Loss. 
        As a result this method will create a new DataFrame in an instance 
        of this class with rows representing attributes and columns representing
        Disparate Impact, Logistic Loss and their ratio for Discriminated and
        Privileged groups
        '''
        
        values = []

        for el in range(X.shape[1]):
            w_d_zero = np.repeat(0.0, self.X.shape[1])
            w_p_zero = np.repeat(0.0, self.X.shape[1])

            w_d_zero[el] = self.w_d[el].copy()
            w_p_zero[el] = self.w_p[el].copy()

            y_d_hat = sigmoid(w_d_zero, X)
            y_p_hat = sigmoid(w_p_zero, X)

            di_d = self._disparate_impact(y_d_hat, s)
            di_p = self._disparate_impact(y_p_hat, s)

            ll_d = self._logistic_loss(y, y_d_hat)
            ll_p = self._logistic_loss(y, y_p_hat)

            values.append([X.columns[el], w_d[el], di_d, ll_d, w_p[el], di_p, ll_p])
    
        ks_bargaining = pd.DataFrame(values, columns=['Attribute', 'Coef_D', 'DI_D', 'LL_D', 'Coef_P', 'DI_P', 'LL_P'])
        ks_bargaining['DI_LL_D'] = ks_bargaining['DI_D']/ks_bargaining['LL_D']
        ks_bargaining['DI_LL_P'] = ks_bargaining['DI_P']/ks_bargaining['LL_P']
    
        ks_bargaining = ks_bargaining.set_index('Attribute')
        
        self.ks_dataset = ks_bargaining
    
        return
    
    def _calculate(self):
        ks_data = self.ks_dataset.copy()
    
        # IF NORMALIZATION IS NEEDED
        if self.normalize:
            ks_data = ks_data[['DI_LL_D', 'DI_LL_P']].T.apply(lambda x: x - min(x), axis=1)
        else:
            ks_data = ks_data[['DI_LL_D', 'DI_LL_P']].T
    
        # MULTIPLY VALUE WITH MAX FROM OTHER ROW
        ks_data = ks_data * np.flip(np.max(ks_data, axis=1)).values[:, np.newaxis]

        # OPTIMIZATION VECTOR PREPARATION (MAX_2 * ROW_1 - MAX_1 * ROW_2)
        ks_opt_data = ks_data.iloc[0, :] - ks_data.iloc[1, :]

        # OPTIMIZATION
        def goal_fun_ks(w_ks, ks_data):
            return -np.sum(ks_data * w_ks)

        def const_w_ks(w_ks):
            return np.sum(w_ks) - 1

        def ks_optimize(ks_data):
            w_ks = np.repeat(0.0, ks_data.shape[0])

            cons = ({'type': 'eq', 'fun': const_w_ks})
            bounds = [(0, 1) for n in w_ks]
            
            np.random.seed(seed=2021)
            model = optimize.minimize(fun=goal_fun_ks, x0=w_ks, args=(ks_data), 
                                      method='SLSQP', constraints=cons, bounds=bounds)

            return model

        self.optimization = ks_optimize(ks_opt_data)
        return
    
    def _final_weights(self):
        w_ks = self.w_d * self.optimization.x + self.w_p * (1 - self.optimization.x)
        self.w = w_ks
        
        return
    
    # ----- FIT AND PREDICT -----
    def fit(self, w_d, w_p, X, y, s):
        '''
        Calculate Kalai-Smorodinsky Fair Logistic Regression solution

        Parameters:
        w_d : numpy.array
            Vector of logistic regression coefficients for discriminated group
        w_p : numpy.array
            Vector of logistic regression coefficients for privileged group
        X : pandas.DataFrame
            Matrix of input attributes
        y : pandas.Series
            Output attribute vector (label)
        s : pandas.Series
            Sensitive attribute vector
            
        Returns:
        KSFairLR : self
            Object of Kalai Smorodinsky Fair Logistic Regression
        '''
    
        # SAVE TO CLASS
        self.w_d = w_d
        self.w_p = w_p
        self.X = X
        self.y = y
        self.s = s
        
        # PERFORM IN ORDER
        self._prepare_ks()
        self._calculate()
        self._final_weights()
        
        return self
    
    def predict(self, X):
        '''
        Perform predictions
        
        Parameters:
        X : pandas.DataFrame
            DataFrame for which predicitons should be created
            
        Returns:
        score : ndarray
            Probabilities of the outcome
        '''
        
        return _sigmoid(X)

# Experiment

In [9]:
data = pd.read_csv('Data/adult_prepared.csv')

In [10]:
y = data['Income']
s = data['Sex']

X = data.drop(['Income', 'Sex'], axis=1)
X = (X - np.mean(X))/np.std(X)

In [11]:
X_d = X.loc[s == 1, :]
X_p = X.loc[s == 0, :]

y_d = y[s == 1]
y_p = y[s == 0]

w = np.repeat(0.0, X.shape[1])

np.random.seed(seed=2021)
w_d = optimize.minimize(fun=logistic_loss, x0=w, args=(X_d, y_d), method='SLSQP').x

np.random.seed(seed=2021)
w_p = optimize.minimize(fun=logistic_loss, x0=w, args=(X_p, y_p), method='SLSQP').x

In [12]:
w_d

array([-0.00979595,  0.36193893,  0.05885222,  6.21990438,  0.25698949,
        0.45681634, -0.07728854,  0.08679764,  0.00304607,  0.13773888,
       -0.06817859, -0.13079843, -0.17905286, -0.08963746,  0.22324022,
        0.10568674, -0.55310772,  0.9963871 , -0.50521498, -0.29795124,
       -0.10585004, -0.35619919,  0.62527459,  0.63950295,  0.06747036,
        0.03012814,  0.12239981, -0.22580896, -0.35939612, -0.20071742,
       -0.02153195,  0.18657878, -0.16904293, -0.0642122 ,  0.40223403,
       -0.02507333,  0.02047668, -0.12699095, -0.0174208 ,  0.11061625,
       -0.04406924,  0.06588862, -0.05125516])

In [13]:
w_p

array([ 0.02381715,  0.19689868,  0.02431283,  6.1106116 ,  0.26765291,
        0.23885482, -0.03481642,  0.06697494,  0.03999298, -0.05262586,
       -0.03610741, -0.07745785, -0.28070703, -0.072653  ,  0.23400059,
        0.09651818, -0.10385168,  0.3179215 , -0.37565187,  0.20953251,
       -0.06172431,  0.06721985, -0.01944622, -0.18117273,  0.19076702,
       -0.22816248, -0.21320206, -0.18291017, -0.14658905,  1.60688179,
        0.2114016 , -0.00113102,  0.01209145,  0.06266838, -0.15361467,
       -0.07775242, -0.01541034, -0.00038542, -0.04108725, -0.0253009 ,
       -0.06237742,  0.00617068, -0.03008206])

In [14]:
performance_score(w_d, X, y, s)

AUC: 0.841676546482703, AUC Disc: 0.9099177749906092, AUC Priv: 0.7887198656722318
LL: 0.9203350532539918, LL Disc: 0.36132870217616464, LL Priv: 1.1966571004067812
Disparate impact 0.3003921311331426


In [15]:
performance_score(w_p, X, y, s)

AUC: 0.8584174949202784, AUC Disc: 0.8589295681091325, AUC Priv: 0.8566341362274587
LL: 0.7850611801547899, LL Disc: 0.993621940289187, LL Priv: 0.6819676534265845
Disparate impact 0.7979329555236878


In [16]:
new_model = KS_fair_solution()

new_model.fit(w_d, w_p, X, y, s)

In [17]:
new_model.optimization

     fun: -0.03720750321064222
     jac: array([-0.00317593, -0.00160082, -0.00310886,  0.        , -0.0029984 ,
       -0.00378368, -0.00314565, -0.00289143, -0.00428216, -0.00490541,
       -0.00311956, -0.0032656 , -0.00120816, -0.00284798,  0.        ,
       -0.0031453 ,  0.00763289,  0.01289409, -0.00432301, -0.01284302,
       -0.00323486,  0.00168116,  0.        ,  0.00658117, -0.00021333,
       -0.00968664, -0.00540319, -0.00354004, -0.00193376, -0.0372075 ,
        0.00045363, -0.00137302, -0.00191114, -0.00302979, -0.00049779,
       -0.00333289, -0.00321337, -0.00168957, -0.00319203, -0.00337384,
       -0.0030111 , -0.00315156, -0.003197  ])
 message: 'Optimization terminated successfully.'
    nfev: 495
     nit: 11
    njev: 11
  status: 0
 success: True
       x: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [18]:
new_model.w

array([ 0.02381715,  0.19689868,  0.02431283,  6.1106116 ,  0.26765291,
        0.23885482, -0.03481642,  0.06697494,  0.03999298, -0.05262586,
       -0.03610741, -0.07745785, -0.28070703, -0.072653  ,  0.23400059,
        0.09651818, -0.10385168,  0.3179215 , -0.37565187,  0.20953251,
       -0.06172431,  0.06721985, -0.01944622, -0.18117273,  0.19076702,
       -0.22816248, -0.21320206, -0.18291017, -0.14658905, -0.20071742,
        0.2114016 , -0.00113102,  0.01209145,  0.06266838, -0.15361467,
       -0.07775242, -0.01541034, -0.00038542, -0.04108725, -0.0253009 ,
       -0.06237742,  0.00617068, -0.03008206])

In [19]:
performance_score(new_model.w, X, y, s)

AUC: 0.8642381666115667, AUC Disc: 0.8731113219172607, AUC Priv: 0.857148500116131
LL: 0.6614496094419803, LL Disc: 0.6106858702764913, LL Priv: 0.6865425986366328
Disparate impact 0.7760250242271908
